# Recommender systems

This notebook, by [felipe.alonso@bbva.com](mailto:felipe.alonso@bbva.com) 
(last version: v5. 02/02/2021)

## Table of Contents

1. [Gathering and analysing data](#load_data) 
2. [Most popular movies](#popular)
3. [Collaborative Filtering](#cf)  
4. [Latent Factor Models](#lfm)  
5. [Exercises (advanced)](#exercises)

In [ ]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

---

<a id='load_data'></a>
# 1. Load data

We will use one the [MovieLens datasets](https://grouplens.org/datasets/movielens/), which consitutes one of the most common datasets for implementing and testing recommender engines. Specifically, we will be using the [Lastest Dataset](https://grouplens.org/datasets/movielens/latest/) (Small). This data set consists of:

* **100836 ratings** across **9742 movies**. 
* Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
* These data were created by **610 users** between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.
* Users were selected at random for inclusion. All selected **users had rated at least 20 movies**. No demographic information is included. Each user is represented by an id, and no other information is provided.
* The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. 
* Only movies with at least one rating or tag are included in the dataset. 

For further information read the [README file](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

## 1.1 Load ratings

<div class  = "alert alert-info"> 
Read the file <b>ratings.csv</b> and save it into a pandas Dataframe. Data is in the path './data/ml-latest-small/' 
</div>

In [ ]:
data_path = './data/ml-latest-small/'

# Hint: pandas read_csv function might be useful

df = # YOUR CODE HERE

# we add a new colum, year, containing the year of the timestamp
df['year'] = df.timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y'))

df.head()

In [ ]:
n_users = df.userId.unique().shape[0]
n_movies = df.movieId.unique().shape[0]
print('#ratings =', df.shape[0])
print('#users = ' + str(n_users) + '\n#movies = ' + str(n_movies))

### 1.1.1 Ratings analytics

Let's go deeper into our data, trying to answer questions like:

1. Which is the average rating for all movies? 
2. And the distribution of ratings? 
3. And the average rating by time (years)?
4. And the average number of rated movies per user? 
5. And the distribution of rated movies per user?

<div class  = "alert alert-info"> 
<b> QUESTION </b>: Show (you can do some drawings if you want) some statistics to understand your data
</div>

In [ ]:
# Let's give some metrics

# 1) Average rating for all times
avg_rating = # YOUR CODE HERE
print('Avg. rating :', avg_rating)

# 2) Rating distribution
#... YOUR CODE HERE
plt.show()

# 3) Average rating by year
#... YOUR CODE HERE
plt.show()

# 4) Average number of rated movies per user
#... YOUR CODE HERE
plt.show()

## 1.2 Load movies

In [ ]:
movies = pd.read_csv(data_path + 'movies.csv',sep=',')

print('Number of rows: ', movies.shape[0])
print('Number of (unique) movies: ',len(movies.movieId.unique()))
movies.tail()

### 1.2.1 Duplicated movieId

Unfortunately, the dataset is not completely clear. **There are a few movies with several `ids`**. Let's find them out

In [ ]:
n_movies_per_id = movies[['movieId','title']].groupby('title').count().sort_values('movieId',ascending=False)
n_movies_per_id.columns = ['n_ids']
n_movies_per_id = n_movies_per_id[n_movies_per_id.n_ids>1]

movieId_to_remove = []
for t in n_movies_per_id.index.values:
    print('The movie:',t,', has these ids: ', movies.loc[movies.title == t].movieId.values)
    movieId_to_remove.append(movies.loc[movies.title == t].movieId.values[-1])

print("\nWe have to remove these indexes values: ", movieId_to_remove)

So we remove the `movies.title` duplicates

In [ ]:
movies = movies.drop_duplicates(subset=['title'],keep='first')

print(movies.shape)
movies.tail()

We should remove also all entries in ratings data where `movieId == movieId_to_remov` (note that there would be other possibilities like changing the `movieId` in ratings data)

In [ ]:
df = df[~df.movieId.isin(movieId_to_remove)]

n_users = df.userId.unique().shape[0]
n_items = df.movieId.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)  )

### 1.2.2 Not rated movies

As you can see, there are **9722 rated movies** (`ratings.csv`), while there are **9737 distinct movies** in the database (`movies.csv`). Let's find the list of movies that have not been rated

In [ ]:
ratedMovieIds = df.movieId.unique()
notRatedMoviesTitles = movies.loc[~movies.movieId.isin(ratedMovieIds),['movieId','title']]

print('There are', len(notRatedMoviesTitles), 'movies with no rating, which are:\n')
print('movieId\t title')
print(notRatedMoviesTitles.values)

## 1.3 reindexing users and movies


#### Movies Ids

Let's move forward. In order to analyze the predicted recommendations, let's create a python dictonary for translating `movieId` to movie titles, and `movieId` to an integer id (`idx`). This way `idx` will vary between 0 and 9137-1 (movies in `movies.csv`).

In [ ]:
movies_id_list = np.unique(movies.movieId.values)

# MovieId to idx
movieId_to_idx = {movieId:idx for idx, movieId in enumerate(movies_id_list)}
movies['movieIdx'] = movies['movieId'].apply(lambda x: movieId_to_idx[x])

plt.scatter(movies.movieIdx,movies.movieId)
plt.xlabel('Idx')
plt.ylabel('MovieId')
plt.show()

In [ ]:
# See the result
movies.head()

#### Modify `ratings` to include new information

In [ ]:
idx_to_title = {idx:title for idx,title in zip(movies.movieIdx.values,movies.title.values)}

df['movieIdx'] = df['movieId'].apply(lambda x: movieId_to_idx[x])
df['title'] = df['movieIdx'].apply(lambda x: idx_to_title[x])

df.head()

As we can see, `year` does not represent the year of release. Let's calculate it

In [ ]:
import re

def extract_year(title):
    
    p = re.compile(r"(?:\((\d{4})\))?\s*$")
    m = p.search(title)
    year = m.group(1)
    
    return year

df['year_release'] = df.title.apply(lambda x:extract_year(x))
df.head()

In [ ]:
df.tail()

#### User Ids

User ids start from 1 to 610, and we will change it to range between 0 a 609 (they are assigned sequentially)

In [ ]:
df.userId = df.userId -1
df.head()

#### Retrieve Movies IDs

Define a function that retrieves all the ids and titles for movies containing 'text' in its title

In [ ]:
def return_movie_id(text, ids):
    """
    Inputs:
    - text: string to be looked for in movies titles
    - ids: dictionary of {id:title}
    
    Return: 
    - A list of (id,title) if text found in titles, and an empty list otherwise.
    """
    
    return [(k, v) for k, v in list(ids.items()) if v.lower().find(text.lower()) > -1]
 

In [ ]:
queryTitle = 'Star Wars'
return_movie_id(queryTitle, idx_to_title)

<div class  = "alert alert-info"> 
<b>QUESTION</b>: in the previous cell, change <tt>queryTitle</tt> to find the index of a movie of your taste. Change also <tt>idx_to_movieTitle</tt> for <tt>movieId_to_movieTitle</tt> and check out the differences.
</div>

---
<a id='popular'></a>
# 2. Most popular movies

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Highest averaged rated movie

## 2.1 Most rated movie

In [ ]:
most_rated_movie = df[['userId','title']].groupby(['title']).count().sort_values('userId',ascending=False)
most_rated_movie.columns = ['popularity_score']

most_rated_movie.head(10)

## 2.2 Highest averaged rated movie

In [ ]:
avg_rating_per_movie = df[['title','rating']].groupby('title').agg(['mean', 'count'])
avg_rating_per_movie.columns = ['avg_rating','n_raters']
avg_rating_per_movie.head()

min_raters = 1
avg_rating_per_movie = avg_rating_per_movie[avg_rating_per_movie.n_raters>=min_raters]
avg_rating_per_movie = avg_rating_per_movie.sort_values(['avg_rating'],ascending=False)

avg_rating_per_movie.head(10)

<div class  = "alert alert-info"> 
<b> QUESTION </b>: Change the value of <tt>min_raters</tt> and re-run the above cell. What happens now? 
</div>

---
<a id='cf'></a>
# 3. Collaborative filtering

## 3.1 Naïve approach

*"People who watched (rated/purchased) this movie (product) also watched (rated/purchased)..."* 

Let's build our co-occurrence matrix

In [ ]:
print('Number of rated movies:', len(df.movieIdx.unique()))
print('Number of movies:', len(movies.movieIdx.unique()))

<div class  = "alert alert-info"> 
<b> QUESTION </b>: Aiming to build a co-occurrence matrix, which should be the dimensions of this matrix? 
</div>

### 3.1.1 Build co-occurence matrix

In [ ]:
# This might be useful
movies_per_user_dict = {user_id:idx.values for user_id,idx in df.groupby('userId')['movieIdx']}

# check movies_per_user_dict (uncomment the following lines if necessary)

#this_user_id = 0
#movies_per_user_dict[this_user_id]
#[idx_to_title[k] for k in movies_per_user_dict[this_user_id]] # retrieve titles 

In [ ]:
N = # YOUR CODE HERE 
print(N)

coMatrix = np.zeros((N, N)) # co-occurrence matrix
for userId, ids in movies_per_user_dict.items():
    for m in ids:
        coMatrix[m,ids] = # YOUR CODE HERE

print(coMatrix.shape)

In [ ]:
coMatrix[0:10,0:10]

### 3.1.2 Predictions using the co-occurrence matrix

Now, let's make predictions!

In [ ]:
queryTitle = 'lambs'
return_movie_id(queryTitle, idx_to_title)

In [ ]:
queryMovieId = 510 # YOUR CODE HERE
print('Let\'s make recommendations for: ', idx_to_title[queryMovieId])
print('')

# Get the corresponding row 
queryAnswer = # YOUR CODE HERE

# Get the highest counts (np.argsort might be useful, in descending order)
queryAnswer = # YOUR CODE HERE

# Get the highest counts (do not select the query movie)
queryAnswer = # YOUR CODE HERE

# let's print out the first 20 recommendations
printAnswer = queryAnswer[0:20]
for i,answerId in enumerate(printAnswer):
    print(i+1,'-', idx_to_title[answerId])

<div class  = "alert alert-info"> 
<b> QUESTION </b>: How accurate are these recommendations? Alternatives?
</div>

### 3.1.3 Jaccard similarity 

What about using the [Jaccard similarity index](https://en.wikipedia.org/wiki/Jaccard_index)?

In [ ]:
i = queryMovieId
jaccardScore = np.zeros(N-1)
for j in range(N-1):
    num = # YOUR CODE HERE 
    dem = # YOUR CODE HERE 
    jaccardScore[j] = float(num)/float(dem) 

queryAnswer = np.argsort(jaccardScore)[::-1] #descending order
queryAnswer = queryAnswer[1:] 

# let's print out the first 20 recommendations
printAnswer = queryAnswer[0:20]
for i,answerId in enumerate(printAnswer):
    print(i+1,'-', idx_to_title[answerId])

## 3.2 Memory-Based Collaborative Filtering (CF)

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-user filtering** and **item-item filtering**. 

* User-user CF: *“Users who are similar to you also liked …”*. A *user-user filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

* Item-Item CF: *“Users who liked this movie also liked …”*. In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. 


First, we need to build our utility matrix. 

In [ ]:
# just as a reminder of the information
df.head(2)

In [ ]:
# build utility matrix
n_items = # YOUR CODE HERE 
n_users = # YOUR CODE HERE 
print(n_users, 'x', n_items)

uMatrix = np.zeros((n_users, n_items)) # utility matrix

for row in df.itertuples():
    user_id = row[1]
    item_id = row[6]
    uMatrix[user_id, item_id] = # YOUR CODE HERE 

print('Dimensions: ', uMatrix.shape)

#### Do some checking ...

In [ ]:
# show uMatrix
print(uMatrix[0:10,0:10])

# check for an specific user
print(movies_per_user_dict[5][0:10])

#### Cosine similarity

Now, we define our similarity function

In [ ]:
def cosineSimilarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)  

So we can now calculate our similarity matrices.

In [ ]:
userSimilarity = cosineSimilarity(uMatrix, kind='user')
print('user-user: ', userSimilarity.shape)

itemSimilarity = cosineSimilarity(uMatrix, kind='item')
print('item-item: ', itemSimilarity.shape)

In [ ]:
# do some checking
print(userSimilarity[0:12,0:12].round(2))
print('')
print(itemSimilarity[0:12,0:12].round(2))

#### Find similar users

In [ ]:
# run this cell
def findKsimilars(rowId,simMatrix,k=5):
    qAnswer = simMatrix[rowId,:]
    qIds = np.argsort(qAnswer)[::-1]
    qValues = simMatrix[rowId,qIds]
    
    return [(qIds[j],qValues[j]) for j in range(1,k+1)]

In [ ]:
queryUser = 0
moreSimilarUsers = findKsimilars(queryUser,userSimilarity,k=5)

print('The more similar users to USER_ID =', queryUser, 'are:\n')

for u,v in moreSimilarUsers:
    print('User',u, 'with a similarity of ', v)

#### Find similar items

In [ ]:
queryTitle = 'mallrats'
return_movie_id(queryTitle, idx_to_title)

In [ ]:
queryMovieId = 510
moreSimilarItems = findKsimilars(queryMovieId,itemSimilarity,k=5)

print('The more similar movies to ', idx_to_title[queryMovieId], 'are:\n')

for item in moreSimilarItems:
    print(idx_to_title[item[0]], 'with a similarity of ', item[1])

<div class = "alert alert-success">
<b>Exercise (advanced - optional):</b> Implement centered cosine similarity metric
</div>

```python
def centeredCosineSimilarity(ratings, kind='user', epsilon=1e-9):
    # YOUR CODE HERE
```

### 3.2.1 Predictions for specific user

In [ ]:
useruserCFpredictions = userSimilarity.dot(uMatrix) / np.array([np.abs(userSimilarity).sum(axis=1)]).T 

In [ ]:
queryUser = 500

queryAnswer = uMatrix[queryUser,:]
queryAnswer = np.argsort(queryAnswer)[::-1] #descending order

print('User_ID: ' + str(queryUser) + ', liked the most:')
print(' ')
# let's print out the first 20 recommendations
printAnswer = queryAnswer[0:11]
for answerId in printAnswer:
    print(uMatrix[queryUser,answerId],'-',idx_to_title[answerId])
    
print('\n... and he/she has not seen ... ')

noWatchedMovies = np.where(uMatrix[queryUser,:]==0)[0]
for m in noWatchedMovies[0:11]:
    print(idx_to_title[m])
    
print('\n... among others ...')

In [ ]:
queryAnswer = useruserCFpredictions[queryUser,noWatchedMovies]
queryAnswer = noWatchedMovies[np.argsort(queryAnswer)[::-1]] #descending order

print('so, it is expected he/she also likes ... ')
print(' ')

printAnswer = queryAnswer[0:11]
for answerId in printAnswer:
    print(idx_to_title[answerId])

### 3.2.1. CF Evaluation

We need a training and a test set

In [ ]:
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split(df, test_size=0.25, random_state=0)

print('Training data: ', trainData.shape)
print('Test data: ', testData.shape)

In [ ]:
uMatrixTraining = np.zeros((n_users, n_items)) # utility matrix
for row in trainData.itertuples():
    uMatrixTraining[row[1], row[6]] = row[3]
    
uMatrixTest = np.zeros((n_users, n_items)) # utility matrix
for row in testData.itertuples():
    idx = movieId_to_idx[row[2]]
    uMatrixTest[row[1], row[6]] = row[3]

print(uMatrixTraining.shape)
print(uMatrixTest.shape)

### user-user

In [ ]:
# we use cosine similarity
userSimilarity = cosineSimilarity(uMatrixTraining, kind='user')
print(userSimilarity.shape)

useruserCFpredictions = userSimilarity.dot(uMatrixTraining) / np.array([np.abs(userSimilarity).sum(axis=1)]).T 

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(mean_squared_error(prediction, ground_truth))

In [ ]:
# let's evaluate
print('User-based CF RMSE: ' + str(rmse(useruserCFpredictions, uMatrixTest)))

In [ ]:
meanUserRating = uMatrixTraining.mean(axis=1)

ratingsDiff = (uMatrixTraining - meanUserRating[:, np.newaxis]) 
userItemCFGlobalpredictions = meanUserRating[:, np.newaxis] + userSimilarity.dot(ratingsDiff) / np.array([np.abs(userSimilarity).sum(axis=1)]).T

In [ ]:
print('User-based CF global baseline RMSE: ' + str(rmse(userItemCFGlobalpredictions, uMatrixTest)))

### item-item

In [ ]:
itemItemCFpredictions = uMatrixTraining.dot(itemSimilarity) / np.array([np.abs(itemSimilarity).sum(axis=1)]) 

print('item-based CF RMSE: ' + str(rmse(itemItemCFpredictions, uMatrixTest)))

---
<a id='lfm'></a>
## 4. Model-based CF or Latent factor models

### 4.1 Singular value decomposition

CF can be formulated in terms of latent factors by approximating the utility matrix using singular value decomposition (SVD).

In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(uMatrixTraining, k = 20)
s_diag_matrix=np.diag(s)
svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(svdPredictions, uMatrixTest)))

In [ ]:
queryAnswer = svdPredictions[queryUser,noWatchedMovies]
queryAnswer = noWatchedMovies[np.argsort(queryAnswer)[::-1]] #descending order

print('so, it is expected he/she also likes ... ')
print(' ')

printAnswer = queryAnswer[0:11]
for answerId in printAnswer:
    print(idx_to_title[answerId])

### 4.2 Alternating Least Squares (ALS)

SVD can be very slow and computationally expensive. Besides, when addressing only the relatively few known entries we are highly prone to overfitting.

An scalable alternative to SVD is ALS, which can include regularization terms to prevent overfitting. We will rename our variable to make them more similar to the ALS notation

In [ ]:
R = uMatrixTraining
T = uMatrixTest

Now we define a “selector” matrix $I$ for the training utility matrix $R$, which will contain 0 if the rating matrix has no rating entry, and 1 if the rating matrix contains an entry. 

In [ ]:
# Index matrix for training data
I = R.copy()
I[I > 0] = 1
I[I == 0] = 0

# Index matrix for test data
I2 = T.copy()
I2[I2 > 0] = 1
I2[I2 == 0] = 0

### ALS algorithm

The ALS algorithm aims to estimate two unknown matrices which, when multiplied together, yield the rating matrix. The loss function you will use is the well-known sum of squared errors. The second term is for regularisation to prevent overfitting

<img src="https://latex.codecogs.com/gif.latex?\underset{Q*&space;,&space;P*}{min}\sum_{(u,i)\epsilon&space;K&space;}(r_{ui}-P_u^TQ_i)^2&plus;\lambda(\left&space;\|&space;Q_i&space;\right&space;\|^2&space;&plus;&space;\left&space;\|&space;P_u&space;\right&space;\|^2)$&space;&space;$" title="\underset{q* , p*}{min}\sum_{(u,i)\epsilon K }(r_{ui}-q_i^Tp_u)^2+\lambda(\left \| q_i \right \|^2 + \left \| p_u \right \|^2)" />

In [ ]:
def alsRmse(I,R,Q,P):
    return np.sqrt(np.sum((I * (R - np.dot(P.T,Q)))**2)/len(R[R > 0]))

In [ ]:
# Algorithm free parameters
lmbda = 0.1     # Regularisation weight
k = 20          # Dimensionality of latent feature space
m, n = R.shape  # Number of users and items
n_epochs = 5   # Number of epochs

# Initialization
P = 3 * np.random.rand(k,m) # Latent user feature matrix
Q = 3 * np.random.rand(k,n) # Latent movie feature matrix
Q[0,:] = R[R != 0].mean(axis=0) # Avg. rating for each movie
E = np.eye(k) # (k x k)-dimensional idendity matrix

This takes a while ....

In [ ]:
train_errors = []
test_errors = []

# Repeat until convergence
for epoch in range(n_epochs):
    # Fix Q and estimate P
    for i, Ii in enumerate(I):
        nui = np.count_nonzero(Ii) # Number of items user i has rated
        if (nui == 0): nui = 1 # Be aware of zero counts!
    
        # Least squares solution
        Ai = np.dot(Q, np.dot(np.diag(Ii), Q.T)) + lmbda * nui * E
        Vi = np.dot(Q, np.dot(np.diag(Ii), R[i].T))
        P[:,i] = np.linalg.solve(Ai,Vi)
        
    # Fix P and estimate Q
    for j, Ij in enumerate(I.T):
        nmj = np.count_nonzero(Ij) # Number of users that rated item j
        if (nmj == 0): nmj = 1 # Be aware of zero counts!
        
        # Least squares solution
        Aj = np.dot(P, np.dot(np.diag(Ij), P.T)) + lmbda * nmj * E
        Vj = np.dot(P, np.dot(np.diag(Ij), R[:,j]))
        Q[:,j] = np.linalg.solve(Aj,Vj)
    
    train_rmse = alsRmse(I,R,Q,P)
    test_rmse = alsRmse(I2,T,Q,P)
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
    
    print("[Epoch %d/%d] train error: %f, test error: %f"%(epoch+1, n_epochs, train_rmse, test_rmse))
    
print("Algorithm converged")

In [ ]:
# Check performance by plotting train and test errors
plt.plot(range(n_epochs), train_errors, marker='o', label='Training Data');
plt.plot(range(n_epochs), test_errors, marker='v', label='Test Data');
plt.title('ALS-WR Learning Curve')
plt.xlabel('Number of Epochs');
plt.ylabel('RMSE');
plt.legend()
plt.grid()
plt.show()

### ALS predictions

In [ ]:
alsPredictions = np.dot(P.T,Q)
svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)

print('SVD RMSE: ' + str(rmse(svdPredictions, uMatrixTest)))
print('ALS RMSE: ' + str(rmse(alsPredictions, uMatrixTest)))

In [ ]:
queryAnswer = alsPredictions[queryUser,noWatchedMovies]
queryAnswer = noWatchedMovies[np.argsort(queryAnswer)[::-1]] #descending order

print('so, it is expected he/she also likes ... ')
print(' ')

printAnswer = queryAnswer[0:11]
for answerId in printAnswer:
    print(idx_to_title[answerId])

---
<a id='exercises'></a>
## 5. Exercises (advanced)


<div class = "alert alert-success">
<b>E1:</b> Implement centered cosine similarity metric in Section 3.2
</div>

<div class = "alert alert-success">
<b>E2:</b> Implement global baseline biased in 3.2: $b_{ui} = \mu + b_u + b_i$
</div>

<div class = "alert alert-success">
<b>E3:</b> Implement k-neighbors in 3.2
</div>